In [40]:
#Import statements
import json
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import datetime
import os
import csv
import time
import ast

In [41]:
#This gets the parent directory of the current directory
current = os.getcwd()
path = Path(current)
twits_folder = str(path.parent)+'\\data\\StockTwits'
twits_folder

'C:\\Users\\Russell\\OneDrive - Marist College\\School\\Spring 2020\\DATA 450 Data Capping\\Repository\\data\\StockTwits'

In [42]:
# The stock Symbols that we will iterate through
# Apple = AAPL
# Amazon = AMZN
# Google = GOOGL
# Microsoft = MSFT
# Dell = DELL
# IBM = IBM
# Intel = INTC
# HP = HPQ
# Facebook = FB
# Cisco Systems = CSCO
# Oracle = ORCL
# HP Enterprise = HPE
# Micron Tech = MU
# DXC Tech = DXC  We have recovered all available historical data March 2017 was IPO
# Thermo Fisher Scientific = TMO

stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"] 

In [43]:
#Gathers the json file and all needed data values.
def collect_Twits(res):
    try:
        
        #
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
               
        #Reorders the columns and gets rid of old symbols column
        df = df[['id','body','created_at','entities.sentiment', 'symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})
        

    #The following loops reformat the symbols column
    #It creates a list of dictionaries, {symbol: "symbol of company mentioned", title: "name of company"}
    dataList=[]
    
    for index in df['symbols']:
        symbolsList = []

        for diction in index:

            symbolsList.append({'symbol' : diction.get('symbol'), 'title' : diction.get('title')})
            
        dataList.append(symbolsList)
        
    df.insert(5, "newSymbols" ,dataList)
    
    df = df[['id','body','created','tag', 'newSymbols']]

    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    #returns the dataframe in correct format
    return df

In [ ]:
while True:
    for symbol in stockSymbol:
    
        #selects the file to add to
        file = twits_folder+'\\{}_twits.csv'.format(symbol)
    
        historicalTwits = pd.read_csv(file)
        oldestID = historicalTwits['id'].iloc[-1]
    
        url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
        response = requests.get(url, params = {'max' :  oldestID}).json()
        
        if response['response']['status'] == 429:
            print("requests exceeded")
            time.sleep(600)
            continue
        
        if not (response['cursor']['more']):
            stockSymbol.remove(symbol)
            continue
    
        tempTwitsDf = collect_Twits(response)

        newHistoricalTwits = historicalTwits.append(tempTwitsDf)
    
        newHistoricalTwits.to_csv(file, index=False)
        time.sleep(15)
        #newHistoricalTwits.to_csv(r'{}_twitsCopy.csv'.format(symbol), index=False)
        time.sleep(15)

c:\users\russell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [39]:
newHistoricalTwits

,id,body,created,tag,newSymbols
0,194214700,"$FB Here we go, hope you bought your 215 2/14 ...",2020-02-10 20:09:42,Bullish,"[{'symbol': 'FB', 'title': 'Facebook'}]"
1,194214529,$FB im intrested $225 looks possible this week...,2020-02-10 20:09:01,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"
2,194214111,"$TSLA I&#39;m happy, but wish he didn&#39;t go...",2020-02-10 20:07:18,none,"[{'symbol': 'FB', 'title': 'Facebook'}, {'symb..."
3,194214042,$FB just put in another Green Candle Day.. lol,2020-02-10 20:07:01,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"
4,194213859,$XT $FB Bulls are killing it. I will probably ...,2020-02-10 20:06:10,none,"[{'symbol': 'FB', 'title': 'Facebook'}, {'symb..."
...,...,...,...,...,...
25,182112467,Discussing $FB #earnings on #OptionBlock at ht...,2019-10-31 17:26:03,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"
26,182112228,$FB weeeeeee.. this sucks,2019-10-31 17:24:23,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"
27,182111862,$FB Jim Cramer on Facebook Political Ads: &#39...,2019-10-31 17:22:08,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"
28,182111685,$FB this was the spookiest Halloween ever,2019-10-31 17:20:53,none,"[{'symbol': 'FB', 'title': 'Facebook'}]"


In [7]:
#start the files and gather most recent data right now

#Do not RUN anymore!!

#for symbol in stockSymbol:
    
    #selects the file to add to
    #file = '{}_twits.csv'.format(symbol)
    
    #historicalTwits = pd.read_csv(file)
    #oldestID = historicalTwits['id'][-1]
    
    #url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
    #response = requests.get(url, params = {'max' :  oldestID}).json()
    
    #response = requests.get(url).json()
    
    #tempTwitsDf = collect_Twits(response)
    
    #tempTwitsDf.to_csv(r'{}_twits.csv'.format(symbol), index=False)

## This is file testing for the new repository

In [3]:

file = twits_folder+'\\{}_twits.csv'.format('AAPL')

In [4]:
file

'C:\\Users\\Russell\\OneDrive - Marist College\\School\\Spring 2020\\DATA 450 Data Capping\\Repository\\data\\StockTwits\\AAPL_twits.csv'

In [5]:
dataFrametesting = pd.read_csv(file)

In [6]:
dataFrametesting

,Unnamed: 0,id,body,created,tag,symbols
0,0,194214777,$AAPL $CRM $AMD $BYND $LULU we all know nothin...,2020-02-10 20:10:04,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
1,1,194214706,"At this rate, we may see 323 today. Go $AAPL",2020-02-10 20:09:43,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
2,2,194214590,$AAPL Tesla now resumed production in China. T...,2020-02-10 20:09:17,Bullish,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
3,3,194214323,$AAPL delightful!,2020-02-10 20:08:11,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
4,4,194214309,$AAPL of course Trump interfered in the DOJ co...,2020-02-10 20:08:08,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
5,5,194214281,$AAPL you just love to see it,2020-02-10 20:07:59,Bullish,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
6,6,194213972,$AAPL and $CTSO winners,2020-02-10 20:06:40,Bullish,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
7,7,194213949,$AAPL REVERSAL! Going to EXPLODE! $$$$$$$$$$$$...,2020-02-10 20:06:34,Bullish,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
8,8,194213931,$AAPL funny how certain people get sensitive w...,2020-02-10 20:06:31,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."
9,9,194213742,"$AAPL $spy get 2nd, 3rd mortgage, you can win ...",2020-02-10 20:05:43,none,"[{'id': 686, 'symbol': 'AAPL', 'title': 'Apple..."


In [7]:
response

{'response': {'status': 200},
 'symbol': {'id': 5715,
  'symbol': 'HPQ',
  'title': 'Hewlett-Packard Company',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 8176},
 'cursor': {'more': True, 'since': 6499416, 'max': 6455327},
 'messages': [{'id': 6499416,
   'body': 'A quick look at $AAPL $HPQ $DELL.  Video posted at http://stks.co/23TY',
   'created_at': '2012-01-24T06:59:49Z',
   'user': {'id': 76669,
    'username': 'smtrader',
    'name': '@smtraderCA on Twitter',
    'avatar_url': 'https://avatars.stocktwits.com/production/76669/thumb-1326107627.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/76669/thumb-1326107627.png',
    'join_date': '2011-05-07',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 1010,
    'following': 0,
    'ideas': 8783,
    'watchlist_stocks_count': 0,
    'like_count': 79,
    'plus_tier': '',
    'premium_room': '',
    'trade_app': False},
   'source': {'id': 1, 'title': 'Sto